In [ ]:
# Install system-level dependencies
!apt-get install -y poppler-utils tesseract-ocr

# Install Python power-tools
!pip install pytesseract pdf2image transformers torch

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 2 not upgraded.
Need to get 186 kB of archives.
After this operation, 697 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.12 [186 kB]
Fetched 186 kB in 1s (246 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 117540 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.12_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.12) ...
Setting up poppler-utils (22.02.0-2ubuntu0.12) ...
Processing triggers for man-db (2.10.2-1) ...


In [ ]:
import re
import os
import pytesseract
from pdf2image import convert_from_path
from transformers import pipeline

In [ ]:
class HandwrittenNoteArchitect:
    def __init__(self):
        print("🚀 Initializing FAST AI Summarizer (DistilBART)...")
        # device=-1 uses CPU. If you enable GPU in Colab, change to device=0
        self.summarizer = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6", device=-1)
        self.headings = []
        self.final_output = ""

    def clean_text(self, text):
        # 1. Remove Page Artifacts (Dynamic Pattern: Word + Number + Random junk)
        # This catches "CHAPTER 12", "PAGE 5", "REPRINT 2025" without naming them
        text = re.sub(r'(?i)(CHAPTER|PAGE|REPRINT|UNIT)\s+[A-Z0-9]+', '', text)

        # 2. Fix OCR "Stutter"
        # (Removes single letters hanging at the start of lines, common in OCR)
        text = re.sub(r'\n[a-zA-Z]\s\n', '\n', text)

        # 3. Standard Character Clean
        text = re.sub(r'[^A-Za-z0-9\s\.,!\?\-]', '', text)

        # NEW: The "Junk I" Eraser
        # Removes 'I' only if it's a standalone character in the middle of a sentence
        text = re.sub(r'\b[I]\b', '', text)

        # 4. The "Whitespace Glue"
        # OCR often puts random newlines in the middle of a sentence.
        # This joins them so the AI sees a continuous thought.
        text = " ".join(text.split())

        return text

    def summarize_block(self, text_block):
        """Prevents IndexError by splitting large text into 400-word bites"""
        words = text_block.split()
        chunk_size = 600  #HERE ITS 400
        summarized_chunks = []

        for i in range(0, len(words), chunk_size):
            chunk = " ".join(words[i:i + chunk_size])
            if len(chunk.strip()) > 30: #HERE ITS 100
                try:
                    summary = self.summarizer(chunk, max_length=80, min_length=30, do_sample=False)
                    summarized_chunks.append(summary[0]['summary_text'])
                except Exception:
                    continue
        return " ".join(summarized_chunks)

    def get_dynamic_headings(self, text):
        # 1. Pattern A: ALL CAPS (Major Chapters)
        major_h = re.findall(r'\n([A-Z\s\.]{6,45})\n', text)

        # 2. NEW Pattern B: Safe Roman Numerals [cite: 53, 54]
        # Must be at start of line, Roman Num, then 1-3 Capitalized words, then END of line
        roman_h = re.findall(r'\n([IVXLCDM]+\.?\s+[A-Z][a-z]+(?:\s[A-Z][a-z]+){0,2})\n', text)

        # 3. Pattern C: Title Case (Minor Sections like Seamounts, Atoll) [cite: 54]
        minor_h = re.findall(r'\n([A-Z][a-z]+(?:\s[A-Z][a-z]+){0,3})\n', text)

        # Combine and clean [cite: 56]
        all_candidates = [h.strip() for h in (major_h + roman_h + minor_h)]

        # 4. Dynamic Filters [cite: 58, 67, 69]
        noise_words = ["CHAPTER", "REPRINT", "PAGE", "UNIT"]
        clean = []
        for h in all_candidates:
            if not h.isdigit() and not any(noise in h.upper() for noise in noise_words):
                # Frequency Filter: Kills "Fundamentals" appearing on every page [cite: 69]
                if all_candidates.count(h) <= 2:
                    clean.append(h)

        return list(dict.fromkeys(clean))

    def process_pdf(self, pdf_path):
        self.final_output = "" # Reset memory for new file

        # 1. Image Conversion
        # Converts PDF pages to images for OCR processing [cite: 75, 76]
        pages = convert_from_path(pdf_path)
        full_text = ""
        for page_img in pages:
            # Extracts raw text using Tesseract engine [cite: 78, 79]
            full_text += f"\n{pytesseract.image_to_string(page_img)}\n"

        # 2. Extract Headings (Loosely Coupled)
        # Identifies All-Caps, Title Case, and Roman Numeral patterns [cite: 81, 51]
        self.headings = self.get_dynamic_headings(full_text)
        print(f"✅ Found {len(self.headings)} Headings")

        # 3. Sectional Processing
        # Iterates through detected headings to slice content chunks [cite: 83, 84]
        for i, curr_h in enumerate(self.headings):
            print(f"✍️ Summarizing: {curr_h}")

            # Defines start and end of the current section [cite: 86, 88]
            start = full_text.find(curr_h) + len(curr_h)
            end = full_text.find(self.headings[i+1]) if i+1 < len(self.headings) else len(full_text)

            # Cleans the sliced text to remove OCR stutter and artifacts [cite: 91, 22]
            content = self.clean_text(full_text[start:end])

            # --- THE CONTENT GATE (Top-Grade Optimization) ---
            # Check if section has enough words to be a real topic (min 25 words) [cite: 42]
            if len(content.split()) > 25:
                # AI processes the chunk to create a summary [cite: 92, 35]
                summary = self.summarize_block(content)
                self.final_output += f"### {curr_h}\n{summary}\n\n"
            else:
                # If tiny (like map labels SH, SA), merge it to previous section without a header
                self.final_output += f" {content} "

        return self.final_output

architect = HandwrittenNoteArchitect()

🚀 Initializing FAST AI Summarizer (DistilBART)...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

Device set to use cpu


In [ ]:
# Change this to your uploaded file name!
PDF_FILE_PATH = "/content/ocean.pdf"

if os.path.exists(PDF_FILE_PATH):
    final_notes = architect.process_pdf(PDF_FILE_PATH)

    # Save as a readable text file
    with open("handwritten_ready_notes.txt", "w") as f:
        f.write(final_notes)

    print("\n✅ SUCCESS! Notes saved in 'handwritten_ready_notes.txt'")
    print("-" * 30)
    print(final_notes[:1000]) # Preview the first 1000 characters
else:
    print(f"❌ Error: Please upload '{PDF_FILE_PATH}' to the /content/ folder.")

✅ Found 20 Headings
✍️ Summarizing: HYDROLOGICAL CYCLE
✍️ Summarizing: RELIEF OF THE OCEAN FLOOR
✍️ Summarizing: TEMPERATURE OF OCEAN WATERS
✍️ Summarizing: SALINITY OF OCEAN WATERS
✍️ Summarizing: SH
✍️ Summarizing: ANTARCTICA
✍️ Summarizing: SA
✍️ Summarizing: Components Processes
✍️ Summarizing: Sublimation
✍️ Summarizing: Continental Shelf
✍️ Summarizing: Continent
Shelf
✍️ Summarizing: Continental Slope
✍️ Summarizing: Deep Sea Plain
✍️ Summarizing: Minor Relief Features


Your max_length is set to 80, but your input_length is only 69. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=34)


✍️ Summarizing: Seamount


Your max_length is set to 80, but your input_length is only 59. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=29)


✍️ Summarizing: Submarine Canyons


Your max_length is set to 80, but your input_length is only 49. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=24)


✍️ Summarizing: Guyots
✍️ Summarizing: Atoll
✍️ Summarizing: Factors Affecting Temperature Distribution


Your max_length is set to 80, but your input_length is only 38. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=19)


✍️ Summarizing: Project Work

✅ SUCCESS! Notes saved in 'handwritten_ready_notes.txt'
------------------------------
### HYDROLOGICAL CYCLE
 The hydrological cycle describes the movement of water on, in, and above the earth . Water also undergoes a cycle from the ocean to land and land to ocean . About 71 per cent of the planetary water is found in the oceans .

### RELIEF OF THE OCEAN FLOOR
 The oceans merge so naturally into one another that it is hard to demarcate them . The floors of the oceans are rugged with the worlds largest mountain ranges, deepest trenches and the largest plains . The ocean floors can be divided into four major divisions i the Continental Shelf ii the Continental Slope ii the Deep Sea Plain iv the Oceanic Deeps .  A mid-oceanic ridge is composed of two chains of mountains separated by a large depression . Seamounts are volcanic in origin and can be 3,000-4,500 m tall . Submarine Canyons are deep valleys, some comparable to the Grand Canyon of the Colorado riv

In [ ]:
from IPython.display import Markdown, display
display(Markdown(final_notes))

### HYDROLOGICAL CYCLE
 The hydrological cycle describes the movement of water on, in, and above the earth . Water also undergoes a cycle from the ocean to land and land to ocean . About 71 per cent of the planetary water is found in the oceans .

### RELIEF OF THE OCEAN FLOOR
 The oceans merge so naturally into one another that it is hard to demarcate them . The floors of the oceans are rugged with the worlds largest mountain ranges, deepest trenches and the largest plains . The ocean floors can be divided into four major divisions i the Continental Shelf ii the Continental Slope ii the Deep Sea Plain iv the Oceanic Deeps .  A mid-oceanic ridge is composed of two chains of mountains separated by a large depression . Seamounts are volcanic in origin and can be 3,000-4,500 m tall . Submarine Canyons are deep valleys, some comparable to the Grand Canyon of the Colorado river .

### TEMPERATURE OF OCEAN WATERS
 This section deals with the spatial and vertical variations of temperature in various oceans . The temperature structure of oceans over middle and low latitudes can be described as a three-layer system from surface to the bottom . The average temperature of surface water of the oceans is about 27 C .  The oceans in the northern hemisphere record relatively higher temperature than in the southern hemisphere . Highest temperature is not recorded at the equator but slightly towards north of it . The average annual temperatures for the northern and southern hemisphere are around 19 C and 16 C respectively .

### SALINITY OF OCEAN WATERS
 Salinity is the term used to define the total content of dissolved salts in sea water Table 12.4. It is calculated as the amount of salt in gm dissolved in 1,000 gm 1 kg of seawater . Salinity of 24.7,, has been considered as the upper limit to demarcate brackish water .

### SH
 The salinity for normal open ocean ranges between 33,, and 37 . In the land locked Red Sea, it is as high as 41,, while in the estuaries and the Arctic, the salinity fluctuates from 0 - 35 . The average salinity of the Atlantic Ocean is around 36 ,, The highest salinity is recorded between 15 and 20 latitudes .

     Water storage Evaporation. in oceans Evapotranspiration ### Sublimation
 Nearly 59 per cent of the water that falls on land returns to the atmosphere through evaporation from over the oceans as well as from other places . The remaining is held as freshwater in glaciers and icecaps, groundwater sources, lakes, soil moisture, atmosphere, streams and within life .

### Continental Shelf
 The continental shelf is the extended margin of each continent occupied by relatively shallow seas and gulfs . It is the shallowest part of the ocean showing an average gradient of 1 or even less . The width of the continental shelves vary from one ocean to another .

   iii the ### Deep Sea Plain
 The continental shelf is the extended margin of each continent occupied by relatively shallow seas and gulfs . It is the shallowest part of the ocean showing an average gradient of 1 or less . The width of the continental shelves vary from one ocean to another .

### Minor Relief Features
 A mid-oceanic ridge is composed of two chains of mountains separated by a large depression . The mountain ranges can have peaks as high as 2,500 m and some even reach above the oceans surface .

### Seamount
 Seamounts are volcanic in origin . These can be 3,000-4,500 m tall . The Emperor seamount, an extension of the Hawaiian Islands in the Pacific Ocean, is a good example .

### Submarine Canyons
 These are deep valleys, some comparable to the Grand Canyon of the Colorado river . They are sometimes found cutting across the continental shelves and slopes, often extending -26 WATER OCEANS from the mouths of large rivers .

### Guyots
 More than 10,000 seamounts and guyots exist in the Pacific Ocean alone . They show evidences of gradual subsidence through stages to become flat topped submerged mountains .

### Atoll
 These are low islands found in the tropical oceans consisting of coral reefs surrounding a central depression . It may be a part of the sea lagoon, or sometimes form enclosing a body of fresh, brackish, or highly saline water .

### Factors Affecting Temperature Distribution
 The temperature of surface water decreases from the equator towards the poles because the amount of insolation decreases poleward . The oceans in the northern hemisphere receive more heat due to their contact with larger extent of land than those in the southern hemisphere . Onshore winds pile up warm water near the coast and this raises the temperature .  Average annual temperatures for the northern and southern hemisphere are around 19 C and 16 C respectively . Figure 12.4 shows the spatial pattern of surface temperature of the oceans . Highest salinity in water bodies Lake Van in Turkey 330,,, Dead Sea 238 ,, Great Salt Lake 220,. -26 .  Figure 12.5 shows the salinity of the Worlds oceans . Salinity at the surface increases by the loss of water to ice or evaporation, or decreased by the input of fresh waters, such as from the rivers . There is a distinct zone called the halocline where salinity increases sharply .

### Project Work
 i Consult the atlas and show ocean floor relief on the outline of the world map . ii Identify the areas of mid oceanic ridges from the Indian Ocean .

